In [1]:
# blue_pulsing_frames.py
import numpy as np
import matplotlib.pyplot as plt
import os

# === CONFIGURATION ===
T = 10
h = 0.01
frame_dt = 0.1
periods = 30
omega = (6 * np.pi) / 10
A = 0.1
epsilon = 0.25

# === FUNCTIONS ===
def a(t): return epsilon * np.sin(omega * t)
def b(t): return 1 - 2 * epsilon * np.sin(omega * t)
def f(x, t): return a(t) * x**2 + b(t) * x
def df_dx(x, t): return 2 * a(t) * x + b(t)

def compute_flow(X, Y, t):
    fx = f(X, t)
    dfx = df_dx(X, t)
    x_dot = -np.pi * A * np.sin(np.pi * fx) * np.cos(np.pi * Y)
    y_dot = np.pi * A * np.cos(np.pi * fx) * np.sin(np.pi * Y) * dfx
    return x_dot, y_dot

def rk4_flow(X, Y, t, h):
    k1x, k1y = compute_flow(X, Y, t)
    k2x, k2y = compute_flow(X + 0.5*h*k1x, Y + 0.5*h*k1y, t + 0.5*h)
    k3x, k3y = compute_flow(X + 0.5*h*k2x, Y + 0.5*h*k2y, t + 0.5*h)
    k4x, k4y = compute_flow(X + h*k3x, Y + h*k3y, t + h)
    X_new = X + (h/6)*(k1x + 2*k2x + 2*k3x + k4x)
    Y_new = Y + (h/6)*(k1y + 2*k2y + 2*k3y + k4y)
    return X_new, Y_new

# === GRID SETUP ===
x_vals = np.linspace(0, 2, 300)
y_vals = np.linspace(0, 1, 150)
X0, Y0 = np.meshgrid(x_vals, y_vals)

# === OUTPUT ===
output_folder = "blue_pulsing"
os.makedirs(output_folder, exist_ok=True)

# --- Compute FTLE for starting particle
X, Y = X0.copy(), Y0.copy()
for t in np.arange(0, T+h, h):
    X, Y = rk4_flow(X, Y, t, h)

dx = x_vals[1] - x_vals[0]
dy = y_vals[1] - y_vals[0]
J11 = (X[2:,1:-1] - X[:-2,1:-1]) / (2*dx)
J12 = (X[1:-1,2:] - X[1:-1,:-2]) / (2*dy)
J21 = (Y[2:,1:-1] - Y[:-2,1:-1]) / (2*dx)
J22 = (Y[1:-1,2:] - Y[1:-1,:-2]) / (2*dy)

ftle = np.full(J11.shape, np.nan)
for i in range(ftle.shape[0]):
    for j in range(ftle.shape[1]):
        J = np.array([[J11[i,j], J12[i,j]],
                      [J21[i,j], J22[i,j]]])
        delta = J.T @ J
        eigvals = np.linalg.eigvalsh(delta)
        max_eig = np.max(eigvals)
        if max_eig > 1e-12 and np.isfinite(max_eig):
            ftle[i,j] = (1.0/T) * np.log(np.sqrt(max_eig))

ftle_full = np.full_like(X0, np.nan)
ftle_full[1:-1,1:-1] = ftle

# --- Find blue region
# Blue region corresponds to min FTLE values
flat_idx = np.nanargmin(ftle_full)
blue_y, blue_x = np.unravel_index(int(flat_idx), ftle_full.shape)
blue_x_val = X0[blue_y, blue_x]
blue_y_val = Y0[blue_y, blue_x]

# === MAIN LOOP: pulsing blue particle
t0_values = np.arange(0, periods*(10/3), frame_dt)
particle_X = np.array([])  # stores all advected particles
particle_Y = np.array([])

for frame_idx, t0 in enumerate(t0_values):
    t_max = t0 + T
    X, Y = X0.copy(), Y0.copy()
    
    # Advect the FTLE field
    for t in np.arange(t0, t_max+h, h):
        X, Y = rk4_flow(X, Y, t, h)
    
    # Add new particle at blue FTLE spot every frame
    if particle_X.size == 0:
        particle_X = np.array([blue_x_val])
        particle_Y = np.array([blue_y_val])
    else:
        particle_X = np.concatenate([particle_X, [blue_x_val]])
        particle_Y = np.concatenate([particle_Y, [blue_y_val]])
    
    # Advect all particles for this frame
    for t in np.arange(t0, t0 + frame_dt + h, h):
        particle_X, particle_Y = rk4_flow(particle_X, particle_Y, t, h)
    
    # FTLE Jacobians
    J11 = (X[2:,1:-1] - X[:-2,1:-1]) / (2*dx)
    J12 = (X[1:-1,2:] - X[1:-1,:-2]) / (2*dy)
    J21 = (Y[2:,1:-1] - Y[:-2,1:-1]) / (2*dx)
    J22 = (Y[1:-1,2:] - Y[1:-1,:-2]) / (2*dy)

    ftle = np.full(J11.shape, np.nan)
    for i in range(ftle.shape[0]):
        for j in range(ftle.shape[1]):
            J = np.array([[J11[i,j], J12[i,j]],
                          [J21[i,j], J22[i,j]]])
            delta = J.T @ J
            eigvals = np.linalg.eigvalsh(delta)
            max_eig = np.max(eigvals)
            if max_eig > 1e-12 and np.isfinite(max_eig):
                ftle[i,j] = (1.0/T) * np.log(np.sqrt(max_eig))

    ftle_full[1:-1,1:-1] = ftle

    # === PLOTTING ===
    plt.figure(figsize=(12,6))
    plt.contourf(X0, Y0, np.ma.masked_invalid(ftle_full), levels=100, cmap='jet')
    plt.colorbar(label='FTLE')
    plt.scatter(particle_X, particle_Y, color='black', s=20, marker='o')
    plt.title(f"Blue Pulsing Particle | t={t0:.1f} → {t0+T:.1f}")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.xlim(0,2)
    plt.ylim(0,1)
    plt.gca().set_aspect('equal')
    filename = os.path.join(output_folder, f"blue_pulse_{frame_idx:04d}.png")
    plt.savefig(filename, dpi=150)
    plt.close()
    print(f"Saved: {filename}")


Saved: blue_pulsing/blue_pulse_0000.png
Saved: blue_pulsing/blue_pulse_0001.png
Saved: blue_pulsing/blue_pulse_0002.png
Saved: blue_pulsing/blue_pulse_0003.png
Saved: blue_pulsing/blue_pulse_0004.png
Saved: blue_pulsing/blue_pulse_0005.png
Saved: blue_pulsing/blue_pulse_0006.png
Saved: blue_pulsing/blue_pulse_0007.png
Saved: blue_pulsing/blue_pulse_0008.png
Saved: blue_pulsing/blue_pulse_0009.png
Saved: blue_pulsing/blue_pulse_0010.png
Saved: blue_pulsing/blue_pulse_0011.png
Saved: blue_pulsing/blue_pulse_0012.png
Saved: blue_pulsing/blue_pulse_0013.png
Saved: blue_pulsing/blue_pulse_0014.png
Saved: blue_pulsing/blue_pulse_0015.png
Saved: blue_pulsing/blue_pulse_0016.png
Saved: blue_pulsing/blue_pulse_0017.png
Saved: blue_pulsing/blue_pulse_0018.png
Saved: blue_pulsing/blue_pulse_0019.png
Saved: blue_pulsing/blue_pulse_0020.png
Saved: blue_pulsing/blue_pulse_0021.png
Saved: blue_pulsing/blue_pulse_0022.png
Saved: blue_pulsing/blue_pulse_0023.png
Saved: blue_pulsing/blue_pulse_0024.png


In [2]:
# blue_pulsing_movie.py
import cv2
import os
from glob import glob

def make_movie_from_frames(
    image_folder='blue_pulsing',
    output_filename='blue_pulsing.mp4',
    fps=10
):
    images = sorted(glob(os.path.join(image_folder, 'blue_pulse_*.png')))
    if not images:
        print(f"❌ No frames found in: {image_folder}")
        return

    first_frame = cv2.imread(images[0])
    height, width, _ = first_frame.shape

    out = cv2.VideoWriter(
        output_filename,
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (width, height)
    )

    for img_path in images:
        frame = cv2.imread(img_path)
        if frame is not None:
            out.write(frame)

    out.release()
    print(f"✅ Movie saved to: {os.path.abspath(output_filename)}")

if __name__ == "__main__":
    make_movie_from_frames()


✅ Movie saved to: /Users/krishitavaghani/blue_pulsing.mp4
